In [46]:
from nba_api.stats.static import players
import pandas as pd
import matplotlib.pyplot as plt
import time
plt.style.use('ggplot')

from nba_api.stats.library.parameters import SeasonAll

player_dict = players.get_players()
nba_players = pd.DataFrame.from_dict(player_dict)

# Use ternary operator or write function 
# Names are case sensitive

# find team Ids
from nba_api.stats.static import teams 
teams = teams.get_teams()
nba_teams = pd.DataFrame.from_dict(teams)

In [47]:
def create_dfs(endpoint, start = 1996, end = 2021, season_type = 'Regular Season'):
    
    acc = []
    counter = 0

    for start_year in range(start, end):
        season = '{}-{}'.format(start_year, str(start_year + 1)[2:])
        print(season)
        acc.append(endpoint
                   (per_mode_detailed = 'Totals', 
                    season = season, season_type_all_star = season_type).get_data_frames()[0]
                   [['PLAYER_ID', 'PLAYER_NAME', 'MIN', 'GP', 'W', 'L', 
                                                     'PTS', 'REB', 'AST', 'BLK', 'STL', 'FGM','FGA','FTA', 'PLUS_MINUS']])
        #get totals over the season instead of averages
#         acc[counter]['tot_PTS'] = acc[counter]['PTS'] * acc[counter]['MIN']
#         acc[counter]['tot_REB'] = acc[counter]['REB'] * acc[counter]['MIN']
#         acc[counter]['tot_AST'] = acc[counter]['AST'] * acc[counter]['MIN']
#         acc[counter]['tot_BLK'] = acc[counter]['BLK'] * acc[counter]['MIN']
#         acc[counter]['tot_STL'] = acc[counter]['STL'] * acc[counter]['MIN']
#         acc[counter]['tot_FGM'] = acc[counter]['FGM'] * acc[counter]['MIN']
#         acc[counter]['tot_FGA'] = acc[counter]['FGA'] * acc[counter]['MIN']
#         acc[counter]['tot_FTA'] = acc[counter]['FTA'] * acc[counter]['MIN']
        
        acc[counter]['TSA'] = acc[counter]['FGA'] + (0.44*acc[counter]['FTA'])
        acc[counter]['TS%'] = acc[counter]['PTS'] / (2 * acc[counter]['TSA'])
        acc[counter]['SEASON'] = season
        counter += 1
        time.sleep(1)
        
    return acc

In [22]:
from nba_api.stats.endpoints import leaguedashplayerclutch as clutch

clutch_data = create_dfs(clutch.LeagueDashPlayerClutch)

1996-97
1997-98
1998-99
1999-00
2000-01
2001-02
2002-03
2003-04
2004-05
2005-06
2006-07
2007-08
2008-09
2009-10
2010-11
2011-12
2012-13
2013-14
2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21


In [4]:
clutch_data_playoffs = create_dfs(clutch.LeagueDashPlayerClutch, season_type = 'Playoffs')

1996-97
1997-98
1998-99
1999-00
2000-01
2001-02
2002-03
2003-04
2004-05
2005-06
2006-07
2007-08
2008-09
2009-10
2010-11
2011-12
2012-13
2013-14
2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21


In [48]:
from nba_api.stats.endpoints import leaguedashplayerstats

player_stats = create_dfs(leaguedashplayerstats.LeagueDashPlayerStats)

1996-97
1997-98
1998-99
1999-00
2000-01
2001-02
2002-03
2003-04
2004-05
2005-06
2006-07
2007-08
2008-09
2009-10
2010-11
2011-12
2012-13
2013-14
2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21


In [50]:
player_stats_playoffs = create_dfs(leaguedashplayerstats.LeagueDashPlayerStats, season_type = 'Playoffs')

1996-97
1997-98
1998-99
1999-00
2000-01
2001-02
2002-03
2003-04
2004-05
2005-06
2006-07
2007-08
2008-09
2009-10
2010-11
2011-12
2012-13
2013-14
2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21


In [8]:
def stack_em(lst):
    stacked = pd.concat([lst[0], lst[1]])
    for x in range(2, len(lst)):
        stacked = pd.concat([stacked, lst[x]])
    return stacked

def plot_stats(df1, df2, stat, ):
    fig, axs = plt.subplots(1,3, tight_layout = True)

    axs[0].hist(df1[stat], bins = num_bins)
    axs[1].hist(df2[stat], bins = num_bins)
    axs[2].hist(df1[stat], bins = num_bins)
    axs[2].hist(df2[stat], bins = num_bins, alpha = 0.65)

    plt.show()

In [63]:
clutch_data[1].sort_values('PTS')

,PLAYER_ID,PLAYER_NAME,MIN,GP,W,L,PTS,REB,AST,BLK,STL,FGM,FGA,FTA,PLUS_MINUS,TSA,TS%,SEASON
0,1024,None,0.000000,1,1,0,0,0,0,0,0,0,0,0,0,0.00,NaN,1997-98
91,32,Clifford Rozier,6.166667,1,0,1,0,0,0,0,0,0,0,0,1,0.00,NaN,1997-98
201,999,Jeff Nordgaard,0.083333,1,0,1,0,0,0,0,0,0,0,0,0,0.00,NaN,1997-98
208,29,Jim McIlvaine,0.400000,1,1,0,0,0,0,0,0,0,0,0,0,0.00,NaN,1997-98
81,356,Chris Morris,4.370000,5,3,2,0,0,0,0,0,0,0,0,3,0.00,NaN,1997-98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,779,Glen Rice,152.481667,42,31,11,117,21,5,4,3,37,72,41,74,90.04,0.649711,1997-98
222,252,Karl Malone,148.111667,42,29,13,124,40,14,2,6,41,75,55,77,99.20,0.625000,1997-98
306,397,Reggie Miller,192.660000,50,34,16,144,22,7,0,6,34,83,69,83,113.36,0.635145,1997-98
344,950,Stephon Marbury,200.045000,44,24,20,153,17,31,1,6,53,117,60,8,143.40,0.533473,1997-98


In [7]:
def aggregate_dfs(dfs):
    stacked = stack_em(dfs)
    grouped = stacked.groupby(['PLAYER_ID', 'PLAYER_NAME'])
    aggregates = grouped.agg(sum)
    #get averages by 36 minutes
    aggregates['PTS'] = aggregates['PTS'] / (aggregates['MIN']/36)
    aggregates['REB'] = aggregates['REB'] / (aggregates['MIN']/36)
    aggregates['AST'] = aggregates['AST'] / (aggregates['MIN']/36)
    aggregates['BLK'] = aggregates['BLK'] / (aggregates['MIN']/36)
    aggregates['STL'] = aggregates['STL'] / (aggregates['MIN']/36)
    aggregates['FGM'] = aggregates['FGM'] / (aggregates['MIN']/36)
    aggregates['FGA'] = aggregates['FGA'] / (aggregates['MIN']/36)
    aggregates['FTA'] = aggregates['FTA'] / (aggregates['MIN']/36)
    aggregates['TSA'] = aggregates['FGA'] + (0.44*aggregates['FTA'])
    aggregates['TS%'] = aggregates['PTS'] / (2 * aggregates['TSA']) 
    
    return aggregates

In [31]:
agg_clutch_data = aggregate_dfs(all_clutch)

In [24]:
all_clutch = clutch_data + clutch_data_playoffs

In [25]:
len(all_clutch)

50

In [26]:
all_players = player_stats + player_stats_playoffs

NameError: name 'player_stats_playoffs' is not defined

In [54]:
agg_player_stats = aggregate_dfs(all_player_stats)

In [53]:
all_player_stats = player_stats + player_stats_playoffs

In [33]:
no_scrubs_clutch_all = agg_clutch_data[agg_clutch_data['MIN'] >= 72]

In [43]:
no_scrubs_clutch_all.sort_values('STL', ascending = False).head(n=5)

,,MIN,GP,W,L,PTS,REB,AST,BLK,STL,FGM,FGA,FTA,PLUS_MINUS,TSA,TS%
PLAYER_ID,PLAYER_NAME,,,,,,,,,,,,,,,
760,Jerome Kersey,124.831667,49,27,22,7.498097,7.209709,1.153553,0.576777,3.172272,2.595495,8.651651,3.172272,-15,10.047451,0.373134
204025,Tim Frazier,116.185000,42,14,28,10.225072,5.577312,4.647760,0.309851,3.098507,2.788656,6.816715,6.506864,-37,9.679735,0.528169
85,Brent Price,74.448333,32,18,14,10.154693,4.352011,5.319125,0.000000,2.901341,2.901341,8.220466,3.868455,4,9.922586,0.511696
2754,Tony Allen,906.473333,380,226,154,12.192306,7.307441,1.350288,0.794287,2.740290,3.931721,7.506012,6.195439,41,10.232005,0.595793
71,Pooh Richardson,118.950000,33,12,21,10.290038,4.842371,2.723834,0.000000,2.723834,3.934426,14.829760,2.118537,-43,15.761917,0.326421


In [38]:
#generating a top 10 for each stat

len(no_scrubs_clutch_all)

1064

In [11]:
clutch_data_playoffs[0]

,PLAYER_ID,PLAYER_NAME,MIN,GP,W,L,PTS,REB,AST,BLK,STL,FGM,FGA,FTA,PLUS_MINUS,TSA,TS%,SEASON
0,243,Aaron McKie,0.163333,1,0,1,0,0,0,0,0,0,0,0,-2,0.00,NaN,1996-97
1,673,Alan Henderson,0.533333,1,0,1,1,0,0,0,0,0,0,2,1,0.88,0.568182,1996-97
2,275,Allan Houston,14.713333,5,3,2,2,0,0,0,0,1,3,0,7,3.00,0.333333,1996-97
3,297,Alonzo Mourning,24.931667,10,5,5,11,4,1,0,0,3,8,7,3,11.08,0.496390,1996-97
4,358,Anfernee Hardaway,2.038333,2,1,1,1,0,0,1,0,0,1,2,-5,1.88,0.265957,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,895,Tyrone Corbin,10.470000,3,1,2,0,2,1,0,0,0,2,0,1,2.00,0.000000,1996-97
119,124,Vlade Divac,5.585000,2,0,2,0,0,0,1,0,0,0,0,-14,0.00,NaN,1996-97
120,702,Voshon Lenard,21.248333,9,4,5,13,3,0,0,0,3,6,7,7,9.08,0.715859,1996-97
121,445,Wesley Person,8.796667,3,2,1,2,3,1,0,0,0,1,2,3,1.88,0.531915,1996-97


In [13]:
agg_clutch_data_all = aggregate_dfs(all_clutch)

In [14]:
agg_clutch_data_all

,,MIN,GP,W,L,PTS,REB,AST,BLK,STL,FGM,FGA,FTA,PLUS_MINUS,TSA,TS%
PLAYER_ID,PLAYER_NAME,,,,,,,,,,,,,,,
2,Byron Scott,106.278333,38,26,12,18.630326,3.048599,1.354933,0.338733,1.354933,5.758464,10.161996,5.419731,70,12.546678,0.742441
3,Grant Long,315.068333,118,53,65,9.826440,5.713046,1.371131,0.457044,1.942436,2.742262,6.969917,5.141742,-40,9.232283,0.532178
7,Dan Schayes,49.395000,25,16,9,18.220468,6.559368,0.000000,0.000000,0.000000,4.372912,7.288187,11.661099,-5,12.419071,0.733568
9,Sedale Threatt,9.521667,4,1,3,11.342552,3.780851,7.561701,0.000000,3.780851,3.780851,15.123403,0.000000,0,15.123403,0.375000
15,Eric Piatkowski,514.556667,209,71,138,15.391891,4.547604,1.819042,0.209889,0.839558,4.127825,10.214618,5.806941,-68,12.769672,0.602674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630256,Jae'Sean Tate,53.946667,20,6,14,10.677212,5.338606,2.001977,1.334652,0.667326,3.336629,8.675235,4.003955,-32,10.436975,0.511509
1630264,Anthony Gill,3.563333,3,2,1,0.000000,20.205800,0.000000,0.000000,0.000000,0.000000,10.102900,0.000000,-8,10.102900,0.000000
1630267,Facundo Campazzo,46.136667,16,11,5,5.462033,3.901452,4.681743,0.780290,3.121162,0.780290,3.901452,3.121162,33,5.274763,0.517751


In [12]:
all_clutch = clutch_data + clutch_data_playoffs

In [109]:
type(all_clutch)

list

In [100]:
all_clutch[0]

,PLAYER_ID,PLAYER_NAME,MIN,GP,W,L,PTS,REB,AST,BLK,STL,FGM,FGA,FTA,PLUS_MINUS,TSA,TS%,SEASON
0,93,None,0.000000,1,1,0,0,0,0,0,0,0,0,0,0,0.00,NaN,1996-97
1,1049,None,0.000000,1,0,1,0,0,0,0,0,0,0,0,0,0.00,NaN,1996-97
2,2179,None,7.360000,5,4,1,0,0,0,0,0,0,0,0,8,0.00,NaN,1996-97
3,920,A.C. Green,114.593333,34,11,23,13,23,1,0,2,4,10,8,-63,13.52,0.480769,1996-97
4,243,Aaron McKie,67.591667,25,11,14,10,7,3,3,3,3,9,2,-15,9.88,0.506073,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,335,Willie Anderson,8.503333,5,4,1,4,2,3,0,0,1,2,2,11,2.88,0.694444,1996-97
392,416,Willie Burton,15.055000,6,5,1,2,3,0,0,0,0,3,2,6,3.88,0.257732,1996-97
393,1365,Xavier McDaniel,49.140000,19,9,10,25,13,3,0,0,9,19,8,-7,22.52,0.555062,1996-97
394,212,Yinka Dare,1.978333,2,1,1,0,0,0,0,0,0,0,0,-5,0.00,NaN,1996-97


In [103]:
len(clutch_data_playoffs)

31

In [106]:
clutch_data_playoffs = clutch_data_playoffs[6:]

In [107]:
clutch_data_playoffs[0]

,PLAYER_ID,PLAYER_NAME,MIN,GP,W,L,PTS,REB,AST,BLK,STL,FGM,FGA,FTA,PLUS_MINUS,TSA,TS%,SEASON
0,243,Aaron McKie,0.163333,1,0,1,0,0,0,0,0,0,0,0,-2,0.00,NaN,1996-97
1,673,Alan Henderson,0.533333,1,0,1,1,0,0,0,0,0,0,2,1,0.88,0.568182,1996-97
2,275,Allan Houston,14.713333,5,3,2,2,0,0,0,0,1,3,0,7,3.00,0.333333,1996-97
3,297,Alonzo Mourning,24.931667,10,5,5,11,4,1,0,0,3,8,7,3,11.08,0.496390,1996-97
4,358,Anfernee Hardaway,2.038333,2,1,1,1,0,0,1,0,0,1,2,-5,1.88,0.265957,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,895,Tyrone Corbin,10.470000,3,1,2,0,2,1,0,0,0,2,0,1,2.00,0.000000,1996-97
119,124,Vlade Divac,5.585000,2,0,2,0,0,0,1,0,0,0,0,-14,0.00,NaN,1996-97
120,702,Voshon Lenard,21.248333,9,4,5,13,3,0,0,0,3,6,7,7,9.08,0.715859,1996-97
121,445,Wesley Person,8.796667,3,2,1,2,3,1,0,0,0,1,2,3,1.88,0.531915,1996-97


In [130]:
stack_em(clutch_data)

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [122]:
clutch_data[0]

,PLAYER_ID,PLAYER_NAME,MIN,GP,W,L,PTS,REB,AST,BLK,STL,FGM,FGA,FTA,PLUS_MINUS,TSA,TS%,SEASON
0,93,None,0.000000,1,1,0,0,0,0,0,0,0,0,0,0,0.00,NaN,1996-97
1,1049,None,0.000000,1,0,1,0,0,0,0,0,0,0,0,0,0.00,NaN,1996-97
2,2179,None,7.360000,5,4,1,0,0,0,0,0,0,0,0,8,0.00,NaN,1996-97
3,920,A.C. Green,114.593333,34,11,23,13,23,1,0,2,4,10,8,-63,13.52,0.480769,1996-97
4,243,Aaron McKie,67.591667,25,11,14,10,7,3,3,3,3,9,2,-15,9.88,0.506073,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,335,Willie Anderson,8.503333,5,4,1,4,2,3,0,0,1,2,2,11,2.88,0.694444,1996-97
392,416,Willie Burton,15.055000,6,5,1,2,3,0,0,0,0,3,2,6,3.88,0.257732,1996-97
393,1365,Xavier McDaniel,49.140000,19,9,10,25,13,3,0,0,9,19,8,-7,22.52,0.555062,1996-97
394,212,Yinka Dare,1.978333,2,1,1,0,0,0,0,0,0,0,0,-5,0.00,NaN,1996-97


In [125]:
swag = pd.concat([clutch_data[0], clutch_data[1]])

In [128]:
swag = pd.concat([swag, clutch_data[2]])

In [129]:
swag

,PLAYER_ID,PLAYER_NAME,MIN,GP,W,L,PTS,REB,AST,BLK,STL,FGM,FGA,FTA,PLUS_MINUS,TSA,TS%,SEASON
0,93,None,0.000000,1,1,0,0,0,0,0,0,0,0,0,0,0.00,NaN,1996-97
1,1049,None,0.000000,1,0,1,0,0,0,0,0,0,0,0,0,0.00,NaN,1996-97
2,2179,None,7.360000,5,4,1,0,0,0,0,0,0,0,0,8,0.00,NaN,1996-97
3,920,A.C. Green,114.593333,34,11,23,13,23,1,0,2,4,10,8,-63,13.52,0.480769,1996-97
4,243,Aaron McKie,67.591667,25,11,14,10,7,3,3,3,3,9,2,-15,9.88,0.506073,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,962,Walter McCarty,23.056667,6,1,5,6,3,4,0,0,1,4,6,-9,6.64,0.451807,1998-99
362,445,Wesley Person,54.106667,19,10,9,15,7,0,0,0,6,24,5,-7,26.20,0.286260,1998-99
363,781,Will Perdue,0.001667,1,1,0,0,0,0,0,0,0,0,0,0,0.00,NaN,1998-99
364,1061,William Cunningham,0.550000,1,0,1,0,0,0,0,0,0,0,0,0,0.00,NaN,1998-99
